# INFO834 : TP 3-4

### Forray Gabriel & Clément Massit

Ce TP nous permettre de voir un ensemble de commandes dans le but de manipuler avec précision MongoDB. Nous verrons quelques script en python et en Node.js pour compléter ces manipulations.

Durant ce TP il s'agit de faire des requêtes afin de manipuler MongoDB. Après avoir testé l'importation des données via le mongoimport puis établit quelques commandes de bases, nous sommes capables de manipuler ces données. Il est temps de réaliser un peu de benchmark...

## 7. Un peu de benchmark 

In [38]:
# Imports

from pymongo import MongoClient
import time
import pandas as pd

In [39]:
# Create the data frame

df = pd.read_csv("communes-departement-region.csv")

In [40]:
# Connect to mongod

client = MongoClient("localhost:27017")
db=client.France

In [41]:
# Measure the time for finding each row by the commune name

start = time.time()

for index, row in df.iterrows():
    db.communes.find({"nom_commune":row['nom_commune']})

end = time.time()

print(end - start)

TypeError: 'Collection' object is not callable. If you meant to call the 'findOne' method on a 'Collection' object it is failing because no such method exists.

Pour rechercher chaque commune depuis la liste de leurs noms, on met **14,16 secondes**

## 8. Quand ça presse et qu’il faut y aller vite
Quand on essaye de créer un index unique sur la colonne code_commune_INSEE :

```bash
    db.communes.createIndex( { code_commune_INSEE: 1 }, { unique: true } )
```

On reçois une erreur car deux éléments on le même code INSEE (le 1015)

Mais on peut créer un index sur la colonne nom_commune de la manière suivante :

```bash
    db.communes.createIndex( { nom_commune : 1 } )
```

On relance alors le programme précédent et on l'exécute en seulement **10,55 secondes**

## 12. Et si on se liait ?

Nous allons récupérer en une suele fois la mailing liste et l'ensemble des utilisateurs. Ceci s'effectue avec ".populate".

Voici le code Node.js de l'utilisation de ".populate".

```js
const mongoose = require('mongoose');
const { Schema } = mongoose;
mongoose.connect('mongodb://localhost:27017/Mailing');

const UserSchema = new mongoose.Schema({
    _id: Schema.Types.ObjectId,
    name: String,
    firstname: String,
    email: String,
});

const ListSchema = new mongoose.Schema({
    _id: Schema.Types.ObjectId,
    name: String,
    users: [{
        type: Schema.Types.ObjectId,
        ref: 'User'
    }]
});

const List = mongoose.model("List", ListSchema);
const User = mongoose.model("User", UserSchema);

module.exports = {
    List, User
}

List.find({"name" : "Mailing-List 1"})
    .populate("users")
    .exec(function (err, res){
        if (err) throw err;
        console.log(res)
    });
  ```

## 13. Montrez patte blanche

L'authentification ne fonctionnait pas au début, mais elle a finit par marcher en changeant de port

## 14. On se ferait bien un petit coup de Map Reduce

Après l'exécution de fonction Map et Reduce suivantes :

```
> var mapFunction = function() { if (this.code_region == 84) emit(this.nom_commune, 1); }
> var reduceFunction = function(nom, index) { return Array.sum(index); }
> db.communes.mapReduce(mapFunction(),reduceFunction(), {out : "map_reduce_example"})
```
On obtient :
```
> db.map_reduce_example.find().pretty()
{ "_id" : "Charix", "value" : 1 }
{ "_id" : "Pierrelongue", "value" : 1 }
{ "_id" : "Ségny", "value" : 1 }
{ "_id" : "Billy", "value" : 1 }
{ "_id" : "Corbas", "value" : 1 }
{ "_id" : "Nangy", "value" : 1 }
{ "_id" : "Mézilhac", "value" : 1 }
{ "_id" : "Chambon-sur-Lignon", "value" : 1 }
{ "_id" : "Poëzat", "value" : 1 }
{ "_id" : "Cézens", "value" : 1 }
{ "_id" : "Bost", "value" : 1 }
{ "_id" : "Sappey-en-Chartreuse", "value" : 1 }
{ "_id" : "Valbonnais", "value" : 1 }
{ "_id" : "Landry", "value" : 1 }
{ "_id" : "Saint-Romain-au-Mont-d'Or", "value" : 1 }
{ "_id" : "Saint-Projet-de-Salers", "value" : 1 }
{ "_id" : "Barsac", "value" : 1 }
{ "_id" : "Coucourde", "value" : 2 }
{ "_id" : "Cayres", "value" : 1 }
{ "_id" : "Saint-Marcellin-en-Forez", "value" : 1 }
```
Ce qui est différent du résultat que l'on devrait avoir 

## 15. Sauvegarde et restauration des données

Après l'exécution des différentes commandes, on s'aperçoit qu'un dossier est présent : *"backup"* où il y a toutes les bases de données sauvegardées depuis MongoDB

## 16. Aggregation

Il est possible de réaliser des aggregations.
Ainsi nous pouvons faire des calculs en utilisant les syntaxes suivantes (liste non exhaustive):
* $group
* $sum
* $min
* $max
* $avg

A titre d'exemple : 

Si l'on veut calculer le nombre de codes postaux par état aux Etats Unis alors :
```
db.cities.aggregate({$group:{_id:"$state",nb_cities:{$sum:1}}},{$sort:{nb_cities:-1}})
```
nous retournera dans le Mongo Shell : 
```
{ "_id" : "TX", "nb_cities" : 1671 }
{ "_id" : "NY", "nb_cities" : 1595 }
{ "_id" : "CA", "nb_cities" : 1516 }
{ "_id" : "PA", "nb_cities" : 1458 }
{ "_id" : "IL", "nb_cities" : 1237 }
{ "_id" : "OH", "nb_cities" : 1007 }
{ "_id" : "MO", "nb_cities" : 994 }
{ "_id" : "IA", "nb_cities" : 922 }
{ "_id" : "MN", "nb_cities" : 882 }
{ "_id" : "MI", "nb_cities" : 876 }
{ "_id" : "VA", "nb_cities" : 816 }
{ "_id" : "KY", "nb_cities" : 809 }
{ "_id" : "FL", "nb_cities" : 804 }
{ "_id" : "WI", "nb_cities" : 716 }
{ "_id" : "KS", "nb_cities" : 715 }
{ "_id" : "NC", "nb_cities" : 705 }
{ "_id" : "IN", "nb_cities" : 676 }
{ "_id" : "WV", "nb_cities" : 656 }
{ "_id" : "GA", "nb_cities" : 635 }
{ "_id" : "OK", "nb_cities" : 586 }
```



## 17. Les triggers

In [37]:
import pymongo

client = pymongo.MongoClient('localhost', 27017)
db = client.Mailing
collection = db.users


change_stream = collection.watch()




OperationFailure: The $changeStream stage is only supported on replica sets, full error: {'ok': 0.0, 'errmsg': 'The $changeStream stage is only supported on replica sets', 'code': 40573, 'codeName': 'Location40573'}